In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from IPython.display import Audio
from ipywidgets import interactive

import numpy as np
import scipy

import soundfile as sf
from utils import *

# Índice


1. [Sinais](#sinais)
    1.1 [Ondas Harmônicas](#ondas)
    1.2 [Sinais compostos](#compostos)
2. [Análise de Sinal](#analise)
    2.1 [Amostragem](#amostragem)
    2.2 [Transformada de Fourier](#Fourier)
    2.3 [Resolução](#resolução)
    2.4 [Curiosidade  - Imagens escondidas](#curiosidade)
3. [Explorações](#exp)
4. [Epílogo](#epi)
5. [Links](#link)

## 1. Sinais 

<a id="sinais"></a>

Quando ligamos o receptor ao microfone do nosso computador e gravamos o sinal que está a ser recebido pela placa de som, o que registamos será semelhante ao que gravámos no seguinte ficheiro:

In [ ]:
Audio("media/vlf_sample.wav") 

Vamos ver que sinal é este:

In [ ]:
input_signal, sample_rate = sf.read("media/vlf_sample.wav")


In [ ]:
setup_graph(title='Amostra de uma gravação de VLF', x_label='tempo (segundos)', y_label='amplitude', fig_size=(14,7))

time_array = np.arange(0, len(input_signal)/sample_rate, 1/sample_rate)
plt.plot(time_array, input_signal)
plt.axis('tight')

O sinal representado na figura anterior é muito complexo. Ele é resultado da soma de muitos sinais distintos detectados pelo nosso receptor. Uns provenientes de emissores militares ou emissores de sinal horário, outros de fontes de ruído domésticas como electrodomésticos, lâmpadas fluorescentes ou até mesmo motores.  

Quando estamos a fazer observações de rádio, regra geral, pretendemos ...

é isolar apenas um dos sinais detectados pelo receptor, de modo a que o possamos estudar melhor. Para isso, temos que escolher uma representação do sinal recebido com a qual sejamos capazes de diferenciar as suas várias componentes, permitindo isolar à posteriori o que se pretende.

Com esse fim em vista, este primeiro guião pretende fazer uma (muito) breve introdução/revisão de alguns conceitos e técnicas de análise de sinal que nos permitirá desenvolver algumas atividades com os sinais gravados de VLF

### 1.1 Ondas harmônicas

<a id="ondas"></a>

Sinais periódicos podem ser descritos, ou aproximados, por ondas harmônicas. As mesmas são descritas por uma expressão da forma: 

$$
x(t)=A \sin (\omega t + \phi)
$$

onde:
* $A$ - amplitude
* $\omega$ - frequência angular $(rad/s)$
* $\phi$ - constante de fase

Associado a uma onda harmônica podemos obter:

* $T=\frac{2\pi}{\omega}$ - período da onda $(s)$
* $f=\frac{1}{T}=\frac{\omega}{2\pi}$ - frequência (número de ciclos por segundo) $(s^{-1},Hz)$


É mais fácil se visualizarmos um exemplo particular:

In [ ]:
freq = 1 #hz - ciclos por segundo
amplitude = 3
time_to_plot = 2 # segundos
sample_rate = 100 # Taxa de amostragem - amostras por segundo - (Nota: mais à frente iremos analisar melhor este parâmetro...)

num_samples = sample_rate * time_to_plot

t = np.linspace(0, time_to_plot, int(time_to_plot*sample_rate), endpoint=False) # tempo

signal = amplitude * np.sin(2*np.pi*freq*t)  

setup_graph(x_label='tempo (segundos)', y_label='amplitude', title=' ', fig_size=(12,6))
plt.plot(t, signal)

In [ ]:
def plot_wave(freq = 1,phase=0,ampl = 3):

    time_to_plot = 2 # segundos
    sample_rate = 200 # Taxa de amostragem 
    num_samples = sample_rate * time_to_plot

    t = np.linspace(0, time_to_plot, int(time_to_plot*sample_rate), endpoint=False) # time variable

    signal = ampl * np.sin(2*np.pi*freq*t+phase)  

    setup_graph(x_label='time (in seconds)', y_label='amplitude', title='time domain', ylim = 5)
    
    plt.plot(t, signal) 

In [ ]:
interactive(plot_wave, freq=(1.0,10.0),phase=(0,2*np.pi),ampl = (1,5))

Vamos ouvir uns senos:

In [ ]:
sample_rate = 44100 # Taxa de amostragem 
time_to_plot = 1.5    # segundos
t = np.linspace(0, time_to_plot, int(time_to_plot*sample_rate), endpoint=False) # time variable

freq = 250  # frequência - 200 Hz
signal1 = np.sin(2*np.pi*freq*t)                # pure sine wave at 250 Hz

Audio(signal1, rate=sample_rate)

In [ ]:
time_to_plot = 1.5    # segundos
t = np.linspace(0, time_to_plot, int(time_to_plot*sample_rate), endpoint=False) # time variable

freq = 500  # frequência 
signal2 = np.sin(2*np.pi*freq*t)                

Audio(signal2, rate=sample_rate)

**Ex.**: Crie $3$ segundos de um sinal sonoro de frequência $f=1500Hz$.

In [ ]:
time_to_plot = 1.5    # seconds
t = np.linspace(0, time_to_plot, int(time_to_plot*sample_rate), endpoint=False) # time variable


freq = 1500  # frequency 
signal3 = np.sin(2*np.pi*freq*t)                

# load a NumPy array
Audio(signal3, rate=sample_rate)

**Ex.**: Crie $2$ segundos de um sinal sonoro de frequência $f=17kHz$.

In [ ]:
time_to_plot = 1.5    # seconds
t = np.linspace(0, time_to_plot, int(time_to_plot*sample_rate), endpoint=False) # time variable


freq = 17000  # frequency 
signal3 = np.sin(2*np.pi*freq*t)                

# load a NumPy array
Audio(signal3, rate=sample_rate)

Vamos escrever os comandos anteriores numa unica função e explorar um pouco mais:

In [ ]:
def audio_wave(f1=220.0):
    sample_rate = 44100 # taxa de amostragem
    time_to_plot = 4    # segundos
    times = np.linspace(0, time_to_plot, int(time_to_plot*sample_rate), endpoint=False) # time variable
    
    signal = np.sin(2*np.pi*f1*times)
    display(Audio(data=signal, rate=sample_rate))
    return signal

In [ ]:
s = interactive(audio_wave, f1=(10.0,20000.0))
display(s)

**Ex.**: Efetue um varrimento no intervalo de frequências anteriores e determine os valores minimos e máximos que o seu ouvido consegue detetar.

### 1.2 Sinais compostos
<a id="compostos"></a>

A soma de vários sinais simples forma um sinal composto. Por exemplo:

In [ ]:
time_to_plot = 3 # segundo
sample_rate = 4410 # amostras por segundo

num_samples = sample_rate * time_to_plot

t = np.linspace(0, time_to_plot, int(time_to_plot*sample_rate), endpoint=False) # time variable


freq1 = 500  # frequency 
freq2 = 730  # frequency 
freq3 = 910  # frequency 

signal1 = 1.5*np.sin(2*np.pi*freq1*t)   
signal2 = 2*np.sin(2*np.pi*freq2*t)                
signal3 = 0.5*np.sin(2*np.pi*freq3*t)                

signal = signal1+signal2+signal3

setup_graph(x_label='tempo (segundos)', y_label='amplitude', title=' ', fig_size=(12,6))
plt.plot(t[0:441], signal[0:441])

In [ ]:
Audio(signal, rate=sample_rate)

Os sons que ouvimos no nosso dia a dia são compostos por muitas frequências diferentes...

Ora vejamos um exemplo

In [ ]:
Audio("media/cow.wav") 

In [ ]:
input_signal, sample_rate = sf.read("media/cow.wav")

In [ ]:
setup_graph(title='Mugir de uma vaca', x_label='tempo (segundos)', y_label='amplitude', fig_size=(12,6))

time_array = np.arange(0, len(input_signal)/sample_rate, 1/sample_rate)
plt.plot(time_array, input_signal)
plt.axis('tight')

Vamos escolher uma janela temporal mais pequena para melhor ver a forma deste sinal sonoro:

In [ ]:
setup_graph(title='Mugir de uma vaca', x_label='time (in seconds)', y_label='amplitude', fig_size=(12,6))

# Gráfico entre os instante de tempo t1 e t2 (segundos)
ti = 0.5
tf = 0.6

# Determinação dos indices correspondentes no vetor contendo o sinal 
i1, i2 = int(ti * sample_rate), int(tf * sample_rate)

plt.plot(time_array[i1:i2], input_signal[i1:i2])

Agora até podemos observar algumas semelhanças com o gráfico do nossso primeiro sinal composto ... Seria bom se conseguissemos saber quais as frequências base que constituem (ou predominam) neste sinal... Será possível?

Iremos ver que sim! 

## 2. Análise de Sinal
<a id="analise"></a>

### 2.1 Amostragem

<a id="amostragem"></a>

Quando estamos a fazer a fazer um registo de sinais acontece que quem está a efectuar observações, vai registar os valores dos sinais recebidos com um computador e este não regista continuamente os valores do sinal, mas sim, efectua registos, espaçados por um dado intervalo de tempo $\Delta$, dos sinais que está a receber. 

A este processo chamamos amostragem.

Vejamos um exemplo. Suponhamos que o nosso detector está a receber uma onda com frequência $f_0=3Hz$ descrita por 
$x(t)= 2\sin(6\pi t)$, mas que só efectuamos registos do valor do sinal de $0.125$ em $0.125$ segundos ($\Delta=0.125s$), ou seja, uma frequência de amostragem de $f_a = \frac{1}{0.125} s= 8Hz$. 

In [ ]:
#--- simulação do sinal contínuo:


freq = 3 #hz - ciclos por segundo
amplitude = 3
time_to_plot = 1 # segundos

n_pontos = 1024 # 

num_samples = n_pontos * time_to_plot

t = np.linspace(0, time_to_plot, int(time_to_plot*n_pontos), endpoint=False) # tempo

signal = amplitude * np.sin(2*np.pi*freq*t)  

setup_graph(x_label='tempo (segundos)', y_label='amplitude', title=' ', fig_size=(12,6))
plt.plot(t, signal)


# ----Determinação dos pontos amostrados do sinal 

sample_rate = 8 # taxa de amostragem

num_samples = sample_rate * time_to_plot

t = np.linspace(0, time_to_plot, int(time_to_plot*sample_rate), endpoint=False) # tempo

amostra = amplitude * np.sin(2*np.pi*freq*t)  

plt.plot(t, amostra, 'o', c='r')

Ou seja, depois de gravarmos $1s$ de deste sinal, ficamos com um conjunto discreto de $8$ pontos com informação do sinal original:

In [ ]:
setup_graph(x_label='tempo (segundos)', y_label='amplitude', title=' ', fig_size=(12,6))

plt.plot(t, amostra, 'o', c='r')

ou seja, um vetor com $8$ entradas:

In [ ]:
amostra

Podemos colocar o problema ao contrário e questionarmos-nos se com este conjunto discreto de pontos conseguimos reconstruir o sinal de onde foram obtidos.

No exemplo anterior, ao conjunto de valores registados podemos de facto associar um conjunto arbitrário de sinusoides. A frequência de amostragem
$$
f_a=\frac{1}{\Delta} s^{-1}
$$
foi de $8Hz$ e por consequência, vamos poder ajustar aos pontos uma qualquer sinosoide que tenha uma frequência igual à frequência $f_0$ mais um múltiplo inteiro da frequência de amostragem $f_a$, como, por exemplo, mostra a figura seguinte:

In [ ]:
# ---- Pontos amostrados do sinal 

sample_rate = 8 # taxa de amostragem

num_samples = sample_rate * time_to_plot

t = np.linspace(0, time_to_plot, int(time_to_plot*sample_rate), endpoint=False) # tempo

amostra = amplitude * np.sin(2*np.pi*freq*t)  
setup_graph(x_label='tempo (segundos)', y_label='amplitude', title=' ', fig_size=(12,6))

plt.plot(t, amostra, 'o', c='r')


# ---- Ajuste de vários sinais aos pontos amostrados

n_pontos = 1024 # 

num_samples = n_pontos * time_to_plot

t = np.linspace(0, time_to_plot, int(time_to_plot*n_pontos), endpoint=False) # tempo

signal = amplitude * np.sin(2*np.pi*freq*t)  

plt.plot(t, signal,'-', c='b', alpha=0.75)

for k in [1,2]:
    signal = amplitude * np.sin(2*np.pi*(freq+k*sample_rate)*t)  

    plt.plot(t, signal,'--', c='b', alpha=0.5)


Se analisarmos com cuidado podemos ver o porquê disto acontecer. Tomemos, por exemplo, o valor da quinta medição da figura anterior. Este registo foi efectuado ao segundo $4 \times \frac{1}{8}= 0.5$, e tem como valor

$$
x(4\Delta)= 2\sin(2\pi \times 3\times  (4 \Delta)).
$$

Agora, se considerarmos que a onda têm uma frequência igual a $f_0+m f_a$, onde $m=0,\pm 1, \pm 2,\ldots$ vamos obter 

$$
\phi(4\Delta)= 2\sin(2\pi(f_0+mf_a)4\Delta)= 2\sin(2\pi f_0 4 \Delta+ 2\pi m f_a 4\Delta),
$$
como $f_a=1 / \Delta$, vemos que $f_a\times \Delta = 1$ e podemos simplificar a expressão anterior obtendo

$$
\phi(4\Delta)=  2\sin(2\pi f_0 4 \Delta+ 4 m 2\pi)=2\sin(2\pi f_0 4 \Delta),
$$
uma vez que a função seno é uma função $2\pi$ periódica. Assim, concluímos que o valor de $\phi(4\Delta)$ é igual ao valor de $x(4\Delta)$. 

É fácil ver que o facto de termos considerado o quinto registo não particulariza em nada o resultado e que generalizando temos que a sequência de valores obtidos por amostragem com uma frequência $f_a Hz$, de uma onda com frequência $f_0Hz$, são indistinguíveis dos de uma amostra obtida de uma onde com frequência $(f_0+mf_a)Hz$, com $m=0,\pm 1, \pm 2,\ldots$, tal como mostra a figura anterior. 

Por outro lado, um resultado muito importante da teoria da análise de sinal, o [teorema de amostragem de Nyquist](https://pt.wikipedia.org/wiki/Teorema_da_amostragem_de_Nyquist%E2%80%93Shannon),  garante que um sinal continuo pode ser completamente recuperado de uma sequência de valores obtidos por amostragem, desde que esta seja efectuada com uma taxa de aquisição duas vezes maior à frequência máxima do sinal registado, $f_a \geqslant 2\times f_{\max}$.
Por outras palavras, o teorema afirma que, ao efectuar amostragem de um sinal com uma frequência de $f_a Hz$, apenas se conseguirá discernir sinais de frequências menores ou iguais a $f_a /2$.

Que o resultado é correcto pode ser observado com um exemplo. Contráriamente ao exemplo anterior, onde a frequência  da onda era inferior a taxa de amostragem, consideremos que efectuamos um registo de uma onda de $8Hz$ com uma taxa de amostragem de $6Hz$. Como facilmente observamos pela figura seguinte:


In [ ]:
#--- simulação do sinal contínuo:


freq = 8 #hz - ciclos por segundo
amplitude = 3
time_to_plot = 1 # segundos

n_pontos = 1024 # 

num_samples = n_pontos * time_to_plot

t = np.linspace(0, time_to_plot, int(time_to_plot*n_pontos), endpoint=False) # tempo

signal = amplitude * np.sin(2*np.pi*freq*t)  

setup_graph(x_label='tempo (segundos)', y_label='amplitude', title=' ', fig_size=(12,6))
plt.plot(t, signal)


# ----Determinação dos pontos amostrados do sinal 

sample_rate = 6 # taxa de amostragem

num_samples = sample_rate * time_to_plot

t = np.linspace(0, time_to_plot, int(time_to_plot*sample_rate), endpoint=False) # tempo

amostra = amplitude * np.sin(2*np.pi*freq*t)  

plt.plot(t, amostra, 'o', c='r')

In [ ]:
setup_graph(x_label='tempo (segundos)', y_label='amplitude', title=' ', fig_size=(12,6))

plt.plot(t, amostra, 'o', c='r')

Comparando com o que foi dito anteriormente, vemos que neste exemplo as condições do teorema não são verificadas e efectivamente temos $8Hz> \frac{6}{2}=3Hz$. Logo, ao reconstruir o sinal, este aparece-nos como tendo sido obtido de uma onda de $2Hz$ que, esta sim, apresenta uma frequência menor que metade da frequência de amostragem, $3Hz$.

In [ ]:
# ---- Pontos amostrados do sinal 

sample_rate = 6 # taxa de amostragem

num_samples = sample_rate * time_to_plot

t = np.linspace(0, time_to_plot, int(time_to_plot*sample_rate), endpoint=False) # tempo

amostra = amplitude * np.sin(2*np.pi*freq*t)  
setup_graph(x_label='tempo (segundos)', y_label='amplitude', title=' ', fig_size=(12,6))

plt.plot(t, amostra, 'o', c='r')


# ---- Ajuste de vários sinais aos pontos amostrados

n_pontos = 1024 # 

num_samples = n_pontos * time_to_plot

t = np.linspace(0, time_to_plot, int(time_to_plot*n_pontos), endpoint=False) # tempo


signal = amplitude * np.sin(2*np.pi*2*t)  

plt.plot(t, signal,'--', c='b', alpha=0.7)


Este limite imposto pela frequência de amostragem está sempre presente em  todas as observações de VLF/LF, uma vez que as placas de som dos computadores actuais só suportam taxas de 44000, 48000, 96000 e 19200 amostras/segundo, permitindo assim frequências máximas de observação de 22000, 24000, 48000 e 96000$Hz$ respectivamente.

### 2.2 Transformada de [Fourier](https://pt.wikipedia.org/wiki/Jean_Baptiste_Joseph_Fourier)

<a id="Fourier"></a>


Suponhamos agora que temos uma amostra de um sinal composto. Por exemplo:

In [ ]:
time_to_plot = 2 # segundos
sample_rate = 1024 # amostras por segundo

num_samples = sample_rate * time_to_plot

t = np.linspace(0, time_to_plot, int(time_to_plot*sample_rate), endpoint=False) # time variable

freq1 = 200  # frequência 1 
freq2 = 350  # frequência 2
freq3 = 500  # frequência 3

sinal1 = 1.5*np.sin(2*np.pi*freq1*t)   
sinal2 = 2*np.sin(2*np.pi*freq2*t)                
sinal3 = 0.5*np.sin(2*np.pi*freq3*t)                

sinal = sinal1+sinal2+sinal3

setup_graph(x_label='tempo (segundos)', y_label='amplitude', title=' ', fig_size=(12,6))
plt.plot(t[0:241], sinal[0:241])

Assim, gerados $3$ segundos de um sinal, a uma taxa de amostragem de $1024Hz$, iremos obter  $2048$ pontos.

In [ ]:
fft_out = np.fft.rfftn(sinal)

fft_mag = np.abs(fft_out)/len(fft_out)
num_samples = len(sinal)

rfreqs = sample_rate/2*np.linspace(0,1,num_samples//2+1)

setup_graph(title='FFT', x_label='Frequência', y_label='Amplitude', fig_size=(12,6))
plt.plot(rfreqs, fft_mag)
plt.axis('tight')

Observando o gráfico anterior podemos constatar que o sinal analisado tem $3$ frequências base de $200$, $350$ e $500Hz$, com amplitudes de $1.5$, $2$, e $0.5$ respetivamente.

Voltemos agora a um sinal mais complexo com frequências a variar no tempo (Ficheiro gerado no site- [audiocheck](https://www.audiocheck.net/audiocheck_dtmf.php)):

In [ ]:
Audio("media/tele_15924.wav") 

Vejamos a sua forma:

In [ ]:
input_signal, sample_rate = sf.read("media/tele_15924.wav")
print('A taxa de amostragem do ficheiro é de %i Hz' %sample_rate)

In [ ]:
len(input_signal)/sample_rate

In [ ]:
time_array = np.arange(0, len(input_signal)/sample_rate, 1/sample_rate)

setup_graph(title='sound', x_label='time (in seconds)', y_label='amplitude', fig_size=(12,6))
plt.plot(time_array, input_signal)
plt.axis('tight')

In [ ]:
input_signal

In [ ]:
fft_out = np.fft.rfft(input_signal)

fft_mag = np.abs(fft_out)/len(fft_out)
num_samples = len(input_signal)

rfreqs=sample_rate/2*np.linspace(0,1,num_samples//2+1)

setup_graph(title='FFT', x_label='Frequência', y_label='Amplitude', fig_size=(12,6))
plt.plot(rfreqs, fft_mag)

O sinal anterior é composto por tons de duas frequências - [DTMF](https://pt.wikipedia.org/wiki/DTMF), da sigla em inglês de “Dual-Tone Multi-Frequency”. Em concreto, podemos constatar que é formado por $5$ tons diferentes, contendo $6$ frequências distintas: $697Hz$, $770Hz$,  $852Hz$, $1209Hz$,	$1336Hz$ e $1477Hz$.

Como aplicámos uma transformada de Fourier à totalidade do sinal, conseguimos identificar as frequências distintas que o sinal contem, mas não nos é possível perceber como variam estas frequências ao longo do tempo

Uma forma de conseguir isso, consiste em aplicar uma transformada de Fourier, não à totalidade do sinal que 
estamos a analisar, mas sim a blocos sucessivos do mesmo.

Para analisarmos as variações de frequência em função do tempo podemos recorrer a um [espetrograma](https://en.wikipedia.org/wiki/Spectrogram) do sinal:

In [ ]:
setup_graph(title='Spectrogram', x_label='tempo(segundos)', y_label='Frequência', fig_size=(12,6))

Sxx, freqs, times, im = plt.specgram(input_signal, Fs=sample_rate, scale='linear', NFFT=256)

Vamos ver o sinal e o seu espetrograma em simultâneo:

In [ ]:
fig = plt.figure(figsize=(12, 8))

ax1 = fig.add_subplot(211)
ax1.set_title('Raw wave of ')
ax1.set_ylabel('Amplitude')

Tempgrav= float(len(input_signal))/float(sample_rate)

ax1.plot(np.arange(0,Tempgrav,1/float(sample_rate)), input_signal)
ax1.autoscale(enable=True, axis='both', tight=True)

ax2 = fig.add_subplot(212)
ax2.imshow(Sxx, aspect='auto', origin='lower', 
           extent=[times.min(), times.max(), freqs.min(), freqs.max()])
ax2.set_yticks(freqs[::10])
ax2.set_xticks(times[::10])
ax2.set_title('Spectrogram of ')
ax2.set_ylabel('Freqs in Hz')
ax2.set_xlabel('Seconds')
ax2.autoscale(enable=True, axis='both', tight=True)
plt.show()

Desta forma conseguimos perceber ...

### 2.3 Resolução

<a id="resolução"></a>

> *Não podemos simultâneamente detalhar a nossa análise no tempo e na frequência. Se pretendermos uma resolução temporal elevada, temos de abdicar de resolução em frequência e vice-versa.*

No espetrograma anterior, definimos uma janela temporal com $256$ pontos. Como o sinal foi gravado com uma taxa de amostragem de $8000 Hz$, cada janela corresponde a:

$$\textrm{resolução temporal}:\;\;\;\frac{\textrm{tamanho da janela}}{\textrm{taxa de amostragem}} = \frac{256}{8000}= 0.032\; 
\textrm{  segundos}$$

O recriproco dá-nos informação sobre a resolução em frequência:

$$\textrm{resolução frequência}:\;\;\;\frac{\textrm{taxa de amostragem}}{\textrm{tamanho da janela}} = \frac{8000}{256}= 31.25\; 
\textrm{  Hz}$$

Assim, podemos ver que a resolução em frequência é inversamente proporcional à resolução no tempo, e logo, nunca podemos aumentar uma sem diminuir a outra. Qual a melhor resolução em tempo/frequência depende do sinal e da aplicação em vista, podendo ser obtida por tentativa e experiência.


Vejamos alguns exemplos:

In [ ]:
setup_graph(title='Spectrogram', x_label='tempo(segundos)', y_label='Frequência', fig_size=(12,6))

Sxx, freqs, times, im = plt.specgram(input_signal, Fs=sample_rate, scale='linear', noverlap=0, NFFT=64)

In [ ]:
setup_graph(title='Spectrogram', x_label='tempo(segundos)', y_label='Frequência', fig_size=(12,6))

Sxx, freqs, times, im = plt.specgram(input_signal, Fs=sample_rate, scale='linear', noverlap=0, NFFT=128)

In [ ]:
setup_graph(title='Spectrogram', x_label='tempo(segundos)', y_label='Frequência', fig_size=(12,6))

Sxx, freqs, times, im = plt.specgram(input_signal, Fs=sample_rate, scale='linear', noverlap=0, NFFT=512)

In [ ]:
setup_graph(title='Spectrogram', x_label='tempo(segundos)', y_label='Frequência', fig_size=(12,6))

Sxx, freqs, times, im = plt.specgram(input_signal, Fs=sample_rate, scale='linear', noverlap=0, NFFT=1024)

In [ ]:
setup_graph(title='Spectrogram', x_label='tempo(segundos)', y_label='Frequência', fig_size=(12,6))

Sxx, freqs, times, im = plt.specgram(input_signal, Fs=sample_rate, scale='linear', noverlap=0, NFFT=2048)

Reparar que quando aumentamos o número de pontos da nossa janela temporal, `NFFT` no função `plt.specgram`, vamos aumentando a resulução em termos de frequência, mas perdendo resolução temporal. No último espetrograma, este efeito leva à perda de informação relevante...

### 2.4 Curiosidade  - Imagens escondidas

<a id="curiosidade"></a>


Imagens escondida na 2ªfaixa do album ["Windowlicker"](https://en.wikipedia.org/wiki/Windowlicker) de Aphex Twin. 

In [ ]:
Audio('media/equation9sec.wav') 

In [ ]:
input_signal, sample_rate = sf.read("media/equation9sec.wav")

In [ ]:
setup_graph(title='Spectrogram', x_label='tempo(segundos)', y_label='Frequência', fig_size=(10,6))

Sxx,freqs, times, im = plt.specgram(np.sum(input_signal, axis=1), Fs=sample_rate, NFFT=512, cmap='YlGnBu_r')

plt.axis('tight')
ax = plt.gca()
ax.set_yscale('log')
ax.set_ylim(100, 22000)

## 3. Explorações

<a id="exp"></a>

Analise as amostras de som seguintes efetuando para cada uma delas um gráfico do espectro e espectrograma. 

Consegue identificar as frequencias predominantes nos repetivos sinais?

### A.

In [ ]:
Audio('media/crickets.wav') 

In [ ]:
input_signal, sample_rate = sf.read("media/crickets.wav")
time_array = np.arange(0, len(input_signal)/sample_rate, 1/sample_rate)
setup_graph(title='sound', x_label='time (in seconds)', y_label='amplitude', fig_size=(12,6))
plt.plot(time_array, input_signal)
plt.axis('tight')

### B.

In [ ]:
Audio('media/housefly.wav') 

In [ ]:
input_signal, sample_rate = sf.read("media/housefly.wav")

time_array = np.arange(0, len(input_signal)/sample_rate, 1/sample_rate)
setup_graph(title='sound', x_label='time (in seconds)', y_label='amplitude', fig_size=(12,6))
plt.plot(time_array, input_signal)
plt.axis('tight')

## 4. Epílogo
<a id="epi"></a>

Agora que já temos mais ferramentas, vamos voltar ao sinal inicial para fazermos uma breve análise do mesmo:

In [ ]:
input_signal, sample_rate = sf.read("media/vlf_sample.wav")

In [ ]:
setup_graph(title='Amostra de uma gravação de VLF', x_label='time (in seconds)', y_label='amplitude', fig_size=(12,6))
time_array = np.arange(0, len(input_signal)/sample_rate, 1/sample_rate)
plt.plot(time_array, input_signal)
plt.axis('tight')

In [ ]:
fft_out = np.fft.rfft(input_signal)

fft_mag = np.abs(fft_out)/len(fft_out)

num_samples = len(input_signal)

rfreqs=sample_rate/2*np.linspace(0,1,num_samples//2+1)

setup_graph(title='FFT', x_label='FFT Bins', y_label='magnitude', fig_size=(12,6))
plt.plot(rfreqs, fft_mag)
plt.axis('tight')

Fazendo o espectrograma do mesmo obtemos:

In [ ]:
setup_graph(title='Spectrogram', x_label='tempo(segundos)', y_label='Frequência', fig_size=(12,6))

Sxx,f, t, im = plt.specgram(input_signal, Fs=sample_rate, scale='linear', NFFT=1024)

Aparentemente ainda não conseguimos observar nada de interessante, mas avancemos para o próximo guião e vamos descobrir...

# 5. Links 

<a id="links"></a>

1. [Sound Analysis with the Fourier Transform and Python](https://github.com/calebmadrigal/FourierTalkOSCON)

2. [Image to Spectrogram](http://www.devrand.org/2013/04/image-to-spectrogram.html)

3. [Hidden images in spectrograms](http://scipython.com/blog/hidden-images-in-spectrograms/)

4. [implement-the-spectrogram-from-scratch](https://fairyonice.github.io/implement-the-spectrogram-from-scratch-in-python.html)

5. [decode-the-dial-up-sounds-using-spectrogram](https://fairyonice.github.io/decode-the-dial-up-sounds-using-spectrogram.html)

6. [Free Engineering Lectures - The Short Time Fourier Transform ](https://www.youtube.com/watch?v=g1_wcbGUcDY&feature=youtu.be&t=713)